In [1]:
import pandas as pd
import copy
from janome.tokenizer import Tokenizer
from gensim.models import word2vec
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.metrics import (accuracy_score, confusion_matrix, f1_score,
                             precision_score, recall_score, roc_auc_score)
from sklearn.model_selection import train_test_split
import requests
import numpy as np
from bs4 import BeautifulSoup
from requests.exceptions import Timeout
import time
import re
import urllib.request
from lxml import html
import openpyxl as op
import math

In [40]:
recommend = pd.read_excel('data/news/20221226.xlsx', sheet_name=0)
access = pd.read_excel('data/news/20221226.xlsx', sheet_name=1)
user = pd.read_excel('data/news/20221226.xlsx', sheet_name=2)
user = user.rename(columns={'ID（ハッシュ値）': 'user'})

In [96]:
log = copy.deepcopy(recommend)  # deep copy

#記事をコンマごとに分ける
for i in range(len(log)):
    log['関連記事一覧'][i] = log['関連記事一覧'][i].split(',')

for i in range(len(log)):
    log['関連記事一覧'][i] = list(filter(len, log['関連記事一覧'][i]))

#URLを数値のみの表示にする
for i in range(len(log)):
    log['クリックURL'][i] = log['クリックURL'][i].split('/articles/')[1]

#最も関連記事数の多い記事を探す
g_list = []
for i in range(len(log)):
    g_list.append(len(log['関連記事一覧'][i]))

#各ジャンル名ををdataframeに格納
num = max(g_list)

for i in range(num):
    log[f'article_{i}'] = ''

for i in range(len(log)):
    for j in range(len(log['関連記事一覧'][i])):
        log[f'article_{j}'][i] = log['関連記事一覧'][i][j]

log.drop('関連記事一覧', axis = 1)

for i in log.filter(regex='^article').columns:
    for j in range(len(log)):
        if log[i][j] != '':
            log[i][j] = log[i][j].split('/articles/')[1]

/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_84003/1473455856.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['関連記事一覧'][i] = log['関連記事一覧'][i].split(',')
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_84003/1473455856.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  log['関連記事一覧'][i] = list(filter(len, log['関連記事一覧'][i]))
/var/folders/fz/x8gfkjkj2y97_snsf3gnnrlr0000gn/T/ipykernel_84003/1473455856.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [137]:
log = log.rename(columns={'ID（ハッシュ値）': 'user'})

User情報の前処理

In [100]:
user = pd.get_dummies(user, columns=['会員タイプ', '性別', '年代', '県内外'], drop_first=True)

In [106]:
user = user.set_index('user')

In [113]:
from sklearn.decomposition import PCA

model_pca = PCA(n_components=5)
vecs_list = model_pca.fit_transform(user)

for i in range(5):
    user[f'user_{i}'] = 0.0

for i in range(len(vecs_list)):
    for j in range(5):
        user[f'user_{j}'][i] = vecs_list[i][j]

In [146]:
user['user_5'] = 1.000000
user = user.filter(regex='^user')

In [147]:
# CSV ファイル (employee.csv) として出力
user.to_csv("user.csv")

In [133]:
log = log.drop(columns=['関連記事一覧', 'イベント数'])

In [148]:
# CSV ファイル (employee.csv) として出力
log.to_csv("log_202212.csv")

In [128]:
# import seaborn as sns
# import matplotlib.pyplot as plt
 
# X = vecs_list[:50,0]
# Y = vecs_list[:50,1]
 
# sns.set(font="Hiragino Maru Gothic Pro")
# plt.figure(figsize=(8, 8))
 
# sns.scatterplot(x=X,y=Y)
 
# for i,(x_name,y_name) in enumerate(zip(X,Y)):
#     plt.annotate(user.index[i],(x_name,y_name))
# plt.show()

In [176]:
user

,user_0,user_1,user_2,user_3,user_4,user_5
user,,,,,,
33097a342f0a0b8c13c20397dd10e231,-0.558331,0.212962,0.950159,0.052789,-0.268737,1.0
a7c481c7359a87ffe2658111be9012fb,-0.558331,0.212962,0.950159,0.052789,-0.268737,1.0
0c022e9d92efbea4ea25f4af4a6f9ea1,-0.655231,0.506967,-0.241503,0.742048,-0.340480,1.0
4658ba82a556e55a602108f5e975215f,-0.585682,0.413368,1.075600,0.130256,-0.325863,1.0
3ec59aae33d6b12d4b9fc23f4acb000a,-0.785879,-0.315186,-0.591448,0.479426,-0.242848,1.0
...,...,...,...,...,...,...
1079d364f4689d99c7651674717de296,0.801445,0.577291,0.262924,0.340565,0.086030,1.0
9d6e38fe2f8a1be3b9abda0e23f2a6e9,-0.543973,0.895273,0.379349,0.274937,0.461924,1.0
b741d78b7344b8d8f7d943d73c4c315f,0.006537,-0.686389,0.558460,-0.103042,-0.158145,1.0


In [175]:
log

,user,クリックURL,article_0,article_1,article_2,article_3,article_4
0,39eb968d16742557ee71fa4b58cd2675,679754,679754,681617,,,
1,dfb52d54832e30685d351a52f17917ad,854421,857888,857889,856872,854421,854413
2,dfb52d54832e30685d351a52f17917ad,891820,891820,875031,876049,886155,886880
3,b7374b15b02593d534d5d390d9778469,877260,868084,877260,847149,883519,874613
4,dd4de9f40f268ba7c67ffb9c124e0afd,871422,871422,871250,868098,862610,850555
...,...,...,...,...,...,...,...
2046,ca4302a0d4de0ad997490894574981dd,874782,874782,874704,439179,,
2047,cf704d8a527c21d6a579c8a785b4709a,874782,874782,874704,439179,,
2048,4afc839b7826785eec9144647d0c7de0,874782,874782,874704,439179,,
2049,93b5102e8f29f34b5ca2abd1fb264365,874782,874782,874704,439179,,


In [165]:
# articles = [894658,	892840,	890924,	887084,	886880]
index_all = {}
article_features = {}

In [167]:
articles_np = np.loadtxt('ar_202212.csv')
articles = {}
for art in articles_np:
    articles[int(art[0])] = [float(x) for x in art[1:]]

In [168]:
i=0
for key in articles:		
    index_all[key] = i
    article_features[i] = articles[key][:]
    i += 1

In [197]:
#set_articlesの動作確認

import imp

with open('Yahoo_recommendation/policy_random.py', "r") as fin:
        source = fin.read()

mod = imp.new_module("mod") #impは今後修正が必要になりそう
    # mod = importlib.new_module("mod") 
exec(source, mod.__dict__)
policy = mod
policy.set_articles(articles)

AttributeError: module 'mod' has no attribute 'set_articles'